In [ ]:
from dotenv import  load_dotenv
import psycopg2
import os

In [ ]:
load_dotenv()

In [ ]:
import pandas as pd
df=pd.read_csv("../data/processed/bank_reviews_with_themes.csv")

In [ ]:
# create bank_id mapping  

bank_ids = {
    "Commercial Bank of Ethiopia":1,
    "Bank of Abyssinia":2,
    "Dashen Bank":3
}
df['bank_id']= df['bank'].map(bank_ids)
df.head(5)

In [ ]:
# prepare the insert query and connect to the database
from psycopg2.extras import execute_batch
insert_query = """
     INSERT INTO review(
     review_id,
     bank_id,
     review,
     rating,
     date,
     sentiment_label,
     sentiment_score,
     source
     )
     VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
     ON CONFLICT (review_id) DO NOTHING;
"""
# iterate over the DataFrame rows and Insert each row into the database 
records=[(
    row.review_id,
    row.bank_id,
    row.review,
    row.rating,
    row.date,
    row.sentiment_label,
    row.sentiment_score,
    row.source
) for index, row in df.iterrows()
]
# execute the insert query for all records in batches
execute_batch(cursor, insert_query, records,page_size=500)
# commit the changes and close the connection to the database
conn.commit()
cursor.close()
conn.close()
print("Data inserted successfully")

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="postgres",
    password=os.getenv("DB_PASSWORD")
)

cursor = conn.cursor()

# 2️⃣ Run COUNT query to get total rows in reviews table
cursor.execute("SELECT COUNT(*) FROM review;")
# 3️⃣ Fetch result from the query
total = cursor.fetchone()[0]

print("Total rows in reviews table:", total)

# 4️⃣ Close connection and cursor
cursor.close()